In [1]:
import pandas as pd
import numpy as np

In [14]:
auctions = pd.read_csv('data/train_auctions.csv.gzip', \
                       dtype={'auction_type_id': 'float64', 'country': 'int64', 'date': 'str', \
                              'devise_id': 'int64', 'platform': np.int8, 'ref_type_id': np.int8, \
                              'source_id': np.int8}, parse_dates= ['date'])
auctions.drop('auction_type_id', axis=1, inplace=True)
auctions.head()
# pasar el dtype porque es pesadisimo!!!!!

,country,date,device_id,platform,ref_type_id,source_id
0,6333597102633388268,2019-03-11 14:18:33.290763,6059599345986491085,1,1,0
1,6333597102633388268,2019-03-11 14:18:34.440157,1115994996230693426,1,1,0
2,6333597102633388268,2019-03-11 14:18:35.862360,7463856250762200923,1,1,0
3,6333597102633388268,2019-03-11 14:18:36.167163,7829815350267792040,1,1,0
4,6333597102633388268,2019-03-11 14:18:37.728590,1448534231953777480,1,1,0


In [6]:
len(auctions)

19571319

In [7]:
#auctions.memory_usage(deep=True)

In [16]:
auctions.dtypes

country                 int64
date           datetime64[ns]
device_id               int64
platform                 int8
ref_type_id              int8
source_id                int8
dtype: object

In [12]:
auctions['platform'].value_counts()

1    15541825
2     4029494
Name: platform, dtype: int64

In [13]:
auctions['source_id'].value_counts()

0    13354597
1     4016739
5     1466494
2      582083
6      151406
Name: source_id, dtype: int64

In [17]:
auctions['country'].value_counts()

6333597102633388268    19571319
Name: country, dtype: int64

In [18]:
auctions['device_id'].value_counts()

633139769114048761     27762
7202276635029175071    23055
7298861376107043945    18188
6302840482782120473    16400
5376802567578262905    16367
8963711959081981780    14362
795159065504552200     12275
6841615430383729819    12077
5384039226444052914    11632
3198179064438296471    11565
519658456951133298     10541
2435544514359302887    10274
8751190726444323885    10256
1835469468095030098    10121
1800317677967270083    10036
9027154272484048876     9958
7574657722181550308     9767
5051610571898428259     9642
320899491737875154      9388
8292793494956750501     9315
201759600719985554      9158
8021195662481548645     9128
7650408579713341483     9020
6392567838173461255     8994
1478017482044456084     8839
6203407103275384250     8823
68016455690089173       8809
8496718680483076445     8682
8491787427844905567     8513
4834359787191717922     8495
                       ...  
5751956180575781757        1
2048572451559832028        1
3744576664345883432        1
73670867056061